In [2]:
import duckdb

con = duckdb.connect(database="./../../database/data.duckdb", read_only=True)

In [6]:
commune = con.table("sise_communes").df()
prelev = con.table("sise_prelevements").df()
res = con.table("sise_resultats").df()